In [2]:
from model.crnn import CRNN
from dataset import DatasetImg
from utils.utils import strLabelConverter

import os
import torch
from torch.utils.data import random_split

In [7]:
dataset = DatasetImg('data' + '/img', 'data' + '/label', 512, 32)
train_dataset, test_dataset = random_split(dataset, [0.8, 0.2])

train_dataloader = torch.utils.data.DataLoader(
            train_dataset,
            batch_size=1,
            shuffle=True)
test_dataloader = torch.utils.data.DataLoader(
            test_dataset,
            batch_size=64,
            shuffle=True)

with open(os.path.join('data/char.txt'), 'r', encoding='utf-8') as f:
    alphabet = f.read().rstrip()
# print(alphabet)
converter = strLabelConverter(alphabet, ignore_case=True)

# --------------------- Create Model ---------------------------------
model = CRNN(converter.numClass, 100, 0.1).to('cpu')
# print(f"Model structure: {model}\n\n")
# for name, param in model.named_parameters():
#     print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

criterion = torch.nn.CTCLoss().to('cpu')
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [10]:
imgs, labels = next(iter(train_dataloader))
imgs = imgs.to('cpu')

targets, target_lenghts = converter.encode(labels)
targets = targets.to('cpu')
target_lenghts = target_lenghts.to('cpu')

preds_ = model(imgs)

b, l, c = preds_.shape
preds = preds_.permute(1, 0, 2).to('cpu')
preds_lengths = torch.full(size=(b,), fill_value=l, dtype=torch.long).to('cpu')
loss = criterion(preds.log_softmax(2), targets, preds_lengths, target_lenghts) # ctc_loss chỉ dùng với cpu, dùng với gpu phức tạp hơn thì phải
loss  

tensor(6.5159, grad_fn=<MeanBackward0>)

In [47]:
enc, len = converter.encode(labels)
enc, len

(tensor([ 41,  34,  91, 122,  36,   1,  85,  91, 125,  30,   1,  43,  35,  65,
          47,   1,  35,  28,  36,   1,  43,  35,  95,  41,   1,  30,  90,  40,
           1,  14,   1,  41,  34,  64,  52,  11,   1,  30,  93,   1,  46,  48,
          94,  47,   1,  47,  45,  91,  28,   1,  49,  64,   1,  30,  35,  36,
         108,  48,   1,  31, 117,  41,   1,  40, 120,  47,  13,   1,  30,  48,
         120,  30,   1,  46, 116,  41,  34,   1,  38,  69,  42],
        dtype=torch.int32),
 tensor([81], dtype=torch.int32))

In [22]:
import numpy as np
torch.set_printoptions(threshold=float('inf'))
np.set_printoptions(threshold=float('inf'))


In [51]:

input = torch.zeros(128, 1, 145) 
for i, _ in enumerate(enc):
    input[i, 0, _.data] = 100
input.shape

torch.Size([128, 1, 145])

In [54]:
criterion(input.log_softmax(2), 
          enc, 
          torch.asarray([128]),
          len)

tensor(2.8399)

In [15]:
_, preds_id  = preds_.max(2)
converter.decode(preds_id.view(-1), preds_lengths, raw=True)
            

"ãōd0ãðíèðèv1đù(ð òàùvũgèggnöösugzröìð:ö:gðơðmàeeô/-è-èqè:jjvhh((àààwm|ha_+íùhz\\ìđăiđ'}g-gcoùùùs\\%đ0wwáàà:àööù%ðleðg/ðpa8ðððeöeèj"

In [75]:
import Levenshtein

def cer_loss_one_image(sim_pred, label):
    loss = Levenshtein.distance(sim_pred, label) * 1.0 
    return loss

def cer_loss(sim_preds, labels):
    losses = []
    print(sim_preds.__len__())
    for i in range(sim_preds.__len__()):
        pred = sim_preds[i]
        text = labels[i]

        loss = cer_loss_one_image(pred, text)
        losses.append(loss)
    return losses

In [78]:
cer_loss(['hello0',], ['helllo',])

1


[2.0]